<a href="https://colab.research.google.com/github/rodrigo77garcia/projeto-integrador-finalIV/blob/master/Segundo_Prototipo_Oficina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Geração de Dados Sintéticos



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- Parâmetros de Simulação ---
N_SERVICOS = 500
DATA_INICIO = datetime(2023, 1, 1)

# Listas de Opções
modelos_bomba = ['Bosch VP44', 'Delphi DPCN', 'Stanadyne DB2', 'Denso HP3', 'Common Rail (Gen. 1)', 'Common Rail (Gen. 2)']
problemas_relatados = ['Falha na Partida', 'Fumaça Excessiva', 'Perda de Potência', 'Marcha Lenta Irregular', 'Vazamento de Diesel']
etapas_manutencao = ['Limpeza e Calibração', 'Troca de Bicos', 'Substituição de Válvulas', 'Reparo Completo', 'Diagnóstico e Ajuste']
tecnicos = ['Técnico A', 'Técnico B', 'Técnico C', 'Técnico D']
status_final = ['Concluído', 'Aguardando Peça', 'Retrabalho']
pecas_custo = {
    'Kit Reparo Basico': 150,
    'Bico Injetor': 450,
    'Válvula Reguladora': 600,
    'Sensor de Pressão': 300,
    'Nenhuma': 0
}

# --- Geração dos Dados ---
np.random.seed(42) # Para reprodutibilidade

# 1. Datas de Entrada e Saída
data_entrada = [DATA_INICIO + timedelta(days=np.random.randint(0, 365), hours=np.random.randint(8, 18), minutes=np.random.randint(0, 60)) for _ in range(N_SERVICOS)]
data_saida = [dt + timedelta(days=np.random.randint(1, 5), hours=np.random.randint(1, 4)) for dt in data_entrada]

# 2. Outras Colunas Aleatórias
df = pd.DataFrame({
    'ID_Servico': range(1000, 1000 + N_SERVICOS),
    'Data_Entrada': data_entrada,
    'Data_Saida': data_saida,
    'Modelo_Bomba': np.random.choice(modelos_bomba, N_SERVICOS, p=[0.2, 0.15, 0.1, 0.15, 0.2, 0.2]),
    'Problema_Relatado': np.random.choice(problemas_relatados, N_SERVICOS),
    'Etapas_Manutencao': np.random.choice(etapas_manutencao, N_SERVICOS),
    'Tecnico_Responsavel': np.random.choice(tecnicos, N_SERVICOS),
    'Status_Final': np.random.choice(status_final, N_SERVICOS, p=[0.85, 0.10, 0.05]),
})

# 3. Peças Utilizadas e Custos (com correlação)
df['Peca_Utilizada'] = np.random.choice(list(pecas_custo.keys()), N_SERVICOS, p=[0.3, 0.25, 0.2, 0.15, 0.1])
df['Custo_Pecas'] = df['Peca_Utilizada'].map(pecas_custo)
df['Custo_Mao_Obra'] = np.random.randint(500, 1500, N_SERVICOS)
df['Valor_Total_Servico'] = df['Custo_Pecas'] + df['Custo_Mao_Obra'] + np.random.randint(200, 800, N_SERVICOS) # Margem de lucro simulada

# 4. Variável-Alvo ML: Reclamação Posterior (Simulando uma tendência)
# Ex: Técnico A tem menos reclamação. Reparos completos em Bosch VP44 tem mais.
prob_base_reclamacao = 0.08
df['Reclamacao_Posterior'] = np.random.rand(N_SERVICOS) < prob_base_reclamacao

# Ajustes de probabilidade para simular tendências
df.loc[df['Tecnico_Responsavel'] == 'Técnico A', 'Reclamacao_Posterior'] = np.random.rand(len(df[df['Tecnico_Responsavel'] == 'Técnico A'])) < 0.03
df.loc[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo'), 'Reclamacao_Posterior'] = np.random.rand(len(df[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo')])) < 0.15

# Conversão para string Booleana (para visualização)
df['Reclamacao_Posterior'] = df['Reclamacao_Posterior'].map({True: 'Sim', False: 'Não'})

# 5. Cálculo do Lead Time (Tempo de Serviço)
df['Lead_Time_Dias'] = (df['Data_Saida'] - df['Data_Entrada']).dt.total_seconds() / (24 * 3600)

print(df.head())
print(f"\nConjunto de dados sintéticos gerado com {len(df)} registros.")

   ID_Servico        Data_Entrada          Data_Saida          Modelo_Bomba  \
0        1000 2023-04-13 11:28:00 2023-04-15 14:28:00            Bosch VP44   
1        1001 2023-09-28 15:20:00 2023-10-01 17:20:00            Bosch VP44   
2        1002 2023-04-13 17:18:00 2023-04-16 19:18:00         Stanadyne DB2   
3        1003 2023-08-03 15:52:00 2023-08-06 16:52:00         Stanadyne DB2   
4        1004 2023-04-10 15:23:00 2023-04-12 18:23:00  Common Rail (Gen. 2)   

        Problema_Relatado         Etapas_Manutencao Tecnico_Responsavel  \
0       Perda de Potência      Diagnóstico e Ajuste           Técnico A   
1     Vazamento de Diesel  Substituição de Válvulas           Técnico A   
2  Marcha Lenta Irregular            Troca de Bicos           Técnico C   
3       Perda de Potência           Reparo Completo           Técnico C   
4        Falha na Partida  Substituição de Válvulas           Técnico D   

  Status_Final     Peca_Utilizada  Custo_Pecas  Custo_Mao_Obra  \
0    Con

Geração de Indicadores Chaves (KPI)

In [ ]:
# --- Indicadores de Gestão de Serviço e Finanças ---

# 1. Transparência/Competitividade: Lead Time Médio
lead_time_medio = df['Lead_Time_Dias'].mean()

# 2. Redução de Reclamações: Taxa Global de Reclamação
taxa_reclamacao_global = (df['Reclamacao_Posterior'] == 'Sim').sum() / N_SERVICOS

# 3. Finanças: Valor Médio de Serviço
valor_medio_servico = df['Valor_Total_Servico'].mean()

# 4. Qualidade/Organização: Status do Estoque (Simulado por Aguardando Peça)
servicos_aguardando_peca = (df['Status_Final'] == 'Aguardando Peça').sum()

# 5. Desempenho do Técnico: Taxa de Reclamação por Técnico
reclamacao_por_tecnico = df.groupby('Tecnico_Responsavel')['Reclamacao_Posterior'].apply(lambda x: (x == 'Sim').sum() / len(x)).sort_values(ascending=False)

# 6. Estoque: Peças Mais Utilizadas (Demanda)
demanda_pecas = df['Peca_Utilizada'].value_counts(normalize=True).head(3) * 100 # Top 3 em %

# --- Apresentação dos Indicadores ---
print("\n" + "="*50)
print("             INDICADORES-CHAVE (KPIs)")
print("="*50)

print(f"1. Tempo Médio de Serviço (Lead Time): \t{lead_time_medio:.2f} dias")
print("-" * 50)
print(f"2. Taxa Global de Reclamação (Objetivo: < 5%): \t{taxa_reclamacao_global:.2%}")
print("-" * 50)
print(f"3. Serviços em Espera (Estoque/Gestão): \t{servicos_aguardando_peca} serviços")
print("-" * 50)
print(f"4. Valor Médio de Serviço: \t\tR$ {valor_medio_servico:,.2f}")
print("-" * 50)
print("5. Taxa de Reclamação por Técnico:")
print(reclamacao_por_tecnico.apply(lambda x: f"{x:.2%}"))
print("-" * 50)
print("6. Top 3 Peças Mais Utilizadas (Demanda de Estoque):")
print(demanda_pecas.apply(lambda x: f"{x:.2f}%"))
print("="*50)


             INDICADORES-CHAVE (KPIs)
1. Tempo Médio de Serviço (Lead Time): 	2.50 dias
--------------------------------------------------
2. Taxa Global de Reclamação (Objetivo: < 5%): 	8.00%
--------------------------------------------------
3. Serviços em Espera (Estoque/Gestão): 	46 serviços
--------------------------------------------------
4. Valor Médio de Serviço: 		R$ 1,828.66
--------------------------------------------------
5. Taxa de Reclamação por Técnico:
Tecnico_Responsavel
Técnico D    11.11%
Técnico C     9.92%
Técnico B     9.17%
Técnico A     2.27%
Name: Reclamacao_Posterior, dtype: object
--------------------------------------------------
6. Top 3 Peças Mais Utilizadas (Demanda de Estoque):
Peca_Utilizada
Kit Reparo Basico     31.80%
Bico Injetor          24.00%
Válvula Reguladora    19.80%
Name: proportion, dtype: object


Preparação para o Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Certifique-se de que o DataFrame 'df' do passo anterior está carregado
# Se você está executando em um novo notebook ou sessão, recarregue o código de geração de dados.

# 1. Definir Features (X) e Target (y)
features = ['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada', 'Lead_Time_Dias', 'Valor_Total_Servico']
X = df[features]
y = df['Reclamacao_Posterior'].map({'Sim': 1, 'Não': 0}) # Converter Target para 1 e 0

# 2. One-Hot Encoding para variáveis categóricas
X_encoded = pd.get_dummies(X, columns=['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada'], drop_first=True)

# 3. Divisão Treino/Teste
# Usaremos 70% dos dados para treinar e 30% para testar
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)

# 4. Normalização (Escalonamento)
# Importante para Redes Neurais. Escala as variáveis numéricas para ter média 0 e desvio padrão 1.
scaler = StandardScaler()

# Identificar colunas numéricas (aquelas que não foram transformadas pelo OHE)
numerical_cols = ['Lead_Time_Dias', 'Valor_Total_Servico']
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Dados prontos para o ML.")
print(f"Dimensão dos dados de treino (Features): {X_train.shape}")

Dados prontos para o ML.
Dimensão dos dados de treino (Features): (350, 18)


Arvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Instanciar e Treinar o Modelo
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42) # Limitamos a profundidade para evitar overfitting
dt_model.fit(X_train, y_train)

# 2. Previsão e Avaliação
y_pred_dt = dt_model.predict(X_test)
y_proba_dt = dt_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print("       MODELO 1: ÁRVORE DE DECISÃO")
print("="*50)
print(f"Acurácia no Teste: {accuracy_score(y_test, y_pred_dt):.4f}")
print("\nRelatório de Classificação (Reclamação):")
print(classification_report(y_test, y_pred_dt))


       MODELO 1: ÁRVORE DE DECISÃO
Acurácia no Teste: 0.9000

Relatório de Classificação (Reclamação):
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       138
           1       0.00      0.00      0.00        12

    accuracy                           0.90       150
   macro avg       0.46      0.49      0.47       150
weighted avg       0.84      0.90      0.87       150



In [ ]:
# Mapear a importância das features
feature_importances_dt = pd.Series(dt_model.feature_importances_, index=X_train.columns)
top_5_dt = feature_importances_dt.nlargest(5)

print("\nTop 5 Fatores que Mais Contribuem para Reclamações (Árvore de Decisão):")
print(top_5_dt)


Top 5 Fatores que Mais Contribuem para Reclamações (Árvore de Decisão):
Valor_Total_Servico                  0.568969
Etapas_Manutencao_Reparo Completo    0.109865
Lead_Time_Dias                       0.078944
Peca_Utilizada_Sensor de Pressão     0.077850
Modelo_Bomba_Common Rail (Gen. 2)    0.067704
dtype: float64


Rede Neurais

In [ ]:
from sklearn.neural_network import MLPClassifier

# 1. Instanciar e Treinar o Modelo
# Estrutura simples: duas camadas ocultas com 10 e 5 neurônios, respectivamente
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=500, random_state=42, solver='adam')
mlp_model.fit(X_train, y_train)

# 2. Previsão e Avaliação
y_pred_mlp = mlp_model.predict(X_test)
y_proba_mlp = mlp_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print("          MODELO 2: REDE NEURAL (MLP)")
print("="*50)
print(f"Acurácia no Teste: {accuracy_score(y_test, y_pred_mlp):.4f}")
print("\nRelatório de Classificação (Reclamação):")
print(classification_report(y_test, y_pred_mlp))


          MODELO 2: REDE NEURAL (MLP)
Acurácia no Teste: 0.9133

Relatório de Classificação (Reclamação):
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       138
           1       0.00      0.00      0.00        12

    accuracy                           0.91       150
   macro avg       0.46      0.50      0.48       150
weighted avg       0.85      0.91      0.88       150



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Saída dos Dados para o Excel

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- 1. Geração do Conjunto de Dados Sintéticos (Repetindo o Setup) ---
N_SERVICOS = 500
DATA_INICIO = datetime(2023, 1, 1)
modelos_bomba = ['Bosch VP44', 'Delphi DPCN', 'Stanadyne DB2', 'Denso HP3', 'Common Rail (Gen. 1)', 'Common Rail (Gen. 2)']
problemas_relatados = ['Falha na Partida', 'Fumaça Excessiva', 'Perda de Potência', 'Marcha Lenta Irregular', 'Vazamento de Diesel']
etapas_manutencao = ['Limpeza e Calibração', 'Troca de Bicos', 'Substituição de Válvulas', 'Reparo Completo', 'Diagnóstico e Ajuste']
tecnicos = ['Técnico A', 'Técnico B', 'Técnico C', 'Técnico D']
status_final = ['Concluído', 'Aguardando Peça', 'Retrabalho']
pecas_custo = {'Kit Reparo Basico': 150, 'Bico Injetor': 450, 'Válvula Reguladora': 600, 'Sensor de Pressão': 300, 'Nenhuma': 0}

np.random.seed(42)

data_entrada = [DATA_INICIO + timedelta(days=np.random.randint(0, 365), hours=np.random.randint(8, 18), minutes=np.random.randint(0, 60)) for _ in range(N_SERVICOS)]
data_saida = [dt + timedelta(days=np.random.randint(1, 5), hours=np.random.randint(1, 4)) for dt in data_entrada]

df = pd.DataFrame({
    'ID_Servico': range(1000, 1000 + N_SERVICOS),
    'Data_Entrada': data_entrada,
    'Data_Saida': data_saida,
    'Modelo_Bomba': np.random.choice(modelos_bomba, N_SERVICOS, p=[0.2, 0.15, 0.1, 0.15, 0.2, 0.2]),
    'Problema_Relatado': np.random.choice(problemas_relatados, N_SERVICOS),
    'Etapas_Manutencao': np.random.choice(etapas_manutencao, N_SERVICOS),
    'Tecnico_Responsavel': np.random.choice(tecnicos, N_SERVICOS),
    'Status_Final': np.random.choice(status_final, N_SERVICOS, p=[0.85, 0.10, 0.05]),
})

df['Peca_Utilizada'] = np.random.choice(list(pecas_custo.keys()), N_SERVICOS, p=[0.3, 0.25, 0.2, 0.15, 0.1])
df['Custo_Pecas'] = df['Peca_Utilizada'].map(pecas_custo)
df['Custo_Mao_Obra'] = np.random.randint(500, 1500, N_SERVICOS)
df['Valor_Total_Servico'] = df['Custo_Pecas'] + df['Custo_Mao_Obra'] + np.random.randint(200, 800, N_SERVICOS)
prob_base_reclamacao = 0.08
df['Reclamacao_Posterior'] = np.random.rand(N_SERVICOS) < prob_base_reclamacao
df.loc[df['Tecnico_Responsavel'] == 'Técnico A', 'Reclamacao_Posterior'] = np.random.rand(len(df[df['Tecnico_Responsavel'] == 'Técnico A'])) < 0.03
df.loc[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo'), 'Reclamacao_Posterior'] = np.random.rand(len(df[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo')])) < 0.15
df['Reclamacao_Posterior'] = df['Reclamacao_Posterior'].map({True: 'Sim', False: 'Não'})
df['Lead_Time_Dias'] = (df['Data_Saida'] - df['Data_Entrada']).dt.total_seconds() / (24 * 3600)

# Criar uma cópia limpa dos dados para a aba Excel
df_dados_sinteticos = df.copy()

# --- 2. Geração dos Indicadores (KPIs) ---
lead_time_medio = df['Lead_Time_Dias'].mean()
taxa_reclamacao_global = (df['Reclamacao_Posterior'] == 'Sim').sum() / N_SERVICOS
valor_medio_servico = df['Valor_Total_Servico'].mean()
servicos_aguardando_peca = (df['Status_Final'] == 'Aguardando Peça').sum()
reclamacao_por_tecnico = df.groupby('Tecnico_Responsavel')['Reclamacao_Posterior'].apply(lambda x: (x == 'Sim').sum() / len(x))
demanda_pecas = df['Peca_Utilizada'].value_counts()

# Estruturar os indicadores em um DataFrame para exportação
kpis = {
    'KPI': [
        'Tempo Médio de Serviço (Dias)',
        'Taxa Global de Reclamação',
        'Valor Médio do Serviço (R$)',
        'Serviços Aguardando Peça (Estoque)',
        'Reclamação - Técnico A',
        'Reclamação - Técnico B',
        'Reclamação - Técnico C',
        'Reclamação - Técnico D',
        'Peça Mais Demandada',
        'Peça Menos Demandada'
    ],
    'Valor': [
        f"{lead_time_medio:.2f}",
        f"{taxa_reclamacao_global:.2%}",
        f"{valor_medio_servico:,.2f}",
        servicos_aguardando_peca,
        f"{reclamacao_por_tecnico.get('Técnico A', 0):.2%}",
        f"{reclamacao_por_tecnico.get('Técnico B', 0):.2%}",
        f"{reclamacao_por_tecnico.get('Técnico C', 0):.2%}",
        f"{reclamacao_por_tecnico.get('Técnico D', 0):.2%}",
        demanda_pecas.index[0],
        demanda_pecas.index[-1]
    ]
}
df_kpis = pd.DataFrame(kpis)


# --- 3. Pré-processamento dos Dados para ML ---
features = ['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada', 'Lead_Time_Dias', 'Valor_Total_Servico']
X = df[features]
y = df['Reclamacao_Posterior'].map({'Sim': 1, 'Não': 0})

X_encoded = pd.get_dummies(X, columns=['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
numerical_cols = ['Lead_Time_Dias', 'Valor_Total_Servico']
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


# --- 4. Modelo: Árvore de Decisão ---
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

# Extrair as regras de decisão (Justificativa Clara)
regras_dt = export_text(dt_model, feature_names=list(X_train.columns))

# Importância das Features (para a Justificativa)
feature_importances_dt = pd.Series(dt_model.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(5)

# Estruturar para Excel
relatorio_dt = pd.DataFrame({
    'Metrica': ['Acurácia (Teste)', 'Precisão (Reclamação)', 'Recall (Reclamação)'],
    'Valor': [
        accuracy_score(y_test, y_pred_dt),
        classification_report(y_test, y_pred_dt, output_dict=True)['1']['precision'],
        classification_report(y_test, y_pred_dt, output_dict=True)['1']['recall']
    ]
})

df_arvore = pd.DataFrame({
    'Tipo': ['Regras de Decisão', 'Importância dos Fatores'],
    'Conteudo': [regras_dt, feature_importances_dt.to_string()]
})


# --- 5. Modelo: Rede Neural ---
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=500, random_state=42, solver='adam')
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_test)

# Estruturar para Excel
relatorio_mlp = pd.DataFrame({
    'Metrica': ['Acurácia (Teste)', 'Precisão (Reclamação)', 'Recall (Reclamação)', 'Estrutura da Rede'],
    'Valor': [
        accuracy_score(y_test, y_pred_mlp),
        classification_report(y_test, y_pred_mlp, output_dict=True)['1']['precision'],
        classification_report(y_test, y_pred_mlp, output_dict=True)['1']['recall'],
        "2 Camadas Ocultas (10 e 5 neurônios)"
    ]
})


# --- 6. Exportação para Excel (Multi-Sheet) ---
nome_arquivo = 'relatorio_oficina_ml.xlsx'

# Cria um objeto ExcelWriter
with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
    # Aba 1: Dados Brutos Sintéticos
    df_dados_sinteticos.to_excel(writer, sheet_name='1. Dados Sintéticos', index=False)

    # Aba 2: Indicadores de Gestão (KPIs)
    df_kpis.to_excel(writer, sheet_name='2. KPIs e Finanças', index=False)

    # Aba 3: Resultados da Árvore de Decisão
    # Adicionar o relatório de métricas
    relatorio_dt.to_excel(writer, sheet_name='3. Arvore Decisão (Regras)', startrow=0, index=False)
    # Adicionar regras e importância (Justificativa Clara)
    df_arvore.to_excel(writer, sheet_name='3. Arvore Decisão (Regras)', startrow=len(relatorio_dt) + 2, index=False)

    # Aba 4: Resultados da Rede Neural
    relatorio_mlp.to_excel(writer, sheet_name='4. Rede Neural (Poder Preditivo)', index=False)

print(f"\nSucesso! O arquivo '{nome_arquivo}' foi gerado com 4 abas, contendo dados, KPIs e resultados dos modelos de Machine Learning.")


Sucesso! O arquivo 'relatorio_oficina_ml.xlsx' foi gerado com 4 abas, contendo dados, KPIs e resultados dos modelos de Machine Learning.


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
!pip install streamlit matplotlib plotly scikit-learn pandas numpy pyngrok

In [ ]:

%%writefile dashboard_oficina.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# =================================================================
# 1. FUNÇÃO DE GERAÇÃO E TREINAMENTO DE DADOS
# =================================================================

@st.cache_resource
def gerar_dados_e_modelos():
    """Gera os dados sintéticos e treina os modelos ML (Árvore e MLP)."""
    N_SERVICOS = 500
    DATA_INICIO = datetime(2023, 1, 1)

    # Listas de Opções (omitido para brevidade)
    modelos_bomba = ['Bosch VP44', 'Delphi DPCN', 'Stanadyne DB2', 'Denso HP3', 'Common Rail (Gen. 1)', 'Common Rail (Gen. 2)']
    problemas_relatados = ['Falha na Partida', 'Fumaça Excessiva', 'Perda de Potência', 'Marcha Lenta Irregular', 'Vazamento de Diesel']
    etapas_manutencao = ['Limpeza e Calibração', 'Troca de Bicos', 'Substituição de Válvulas', 'Reparo Completo', 'Diagnóstico e Ajuste']
    tecnicos = ['Técnico A', 'Técnico B', 'Técnico C', 'Técnico D']
    status_final = ['Concluído', 'Aguardando Peça', 'Retrabalho']
    pecas_custo = {'Kit Reparo Basico': 150, 'Bico Injetor': 450, 'Válvula Reguladora': 600, 'Sensor de Pressão': 300, 'Nenhuma': 0}

    np.random.seed(42)

    data_entrada = [DATA_INICIO + timedelta(days=np.random.randint(0, 365), hours=np.random.randint(8, 18), minutes=np.random.randint(0, 60)) for _ in range(N_SERVICOS)]
    data_saida = [dt + timedelta(days=np.random.randint(1, 5), hours=np.random.randint(1, 4)) for dt in data_entrada]

    df = pd.DataFrame({
        'ID_Servico': range(1000, 1000 + N_SERVICOS),
        'Data_Entrada': data_entrada,
        'Data_Saida': data_saida,
        'Modelo_Bomba': np.random.choice(modelos_bomba, N_SERVICOS, p=[0.2, 0.15, 0.1, 0.15, 0.2, 0.2]),
        'Problema_Relatado': np.random.choice(problemas_relatados, N_SERVICOS),
        'Etapas_Manutencao': np.random.choice(etapas_manutencao, N_SERVICOS),
        'Tecnico_Responsavel': np.random.choice(tecnicos, N_SERVICOS),
        'Status_Final': np.random.choice(status_final, N_SERVICOS, p=[0.85, 0.10, 0.05]),
    })

    df['Peca_Utilizada'] = np.random.choice(list(pecas_custo.keys()), N_SERVICOS, p=[0.3, 0.25, 0.2, 0.15, 0.1])
    df['Custo_Pecas'] = df['Peca_Utilizada'].map(pecas_custo)
    df['Custo_Mao_Obra'] = np.random.randint(500, 1500, N_SERVICOS)
    df['Valor_Total_Servico'] = df['Custo_Pecas'] + df['Custo_Mao_Obra'] + np.random.randint(200, 800, N_SERVICOS)

    # Simulação da variável-alvo (Reclamação)
    prob_base_reclamacao = 0.08
    df['Reclamacao_Posterior'] = np.random.rand(N_SERVICOS) < prob_base_reclamacao
    df.loc[df['Tecnico_Responsavel'] == 'Técnico A', 'Reclamacao_Posterior'] = np.random.rand(len(df[df['Tecnico_Responsavel'] == 'Técnico A'])) < 0.03
    df.loc[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo'), 'Reclamacao_Posterior'] = np.random.rand(len(df[(df['Modelo_Bomba'] == 'Bosch VP44') & (df['Etapas_Manutencao'] == 'Reparo Completo')])) < 0.15

    df['Reclamacao_Posterior_Binario'] = df['Reclamacao_Posterior'].map({True: 1, False: 0})
    df['Reclamacao_Posterior'] = df['Reclamacao_Posterior'].map({True: 'Sim', False: 'Não'})
    df['Lead_Time_Dias'] = (df['Data_Saida'] - df['Data_Entrada']).dt.total_seconds() / (24 * 3600)

    # --- Treinamento dos Modelos ---
    features = ['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada', 'Lead_Time_Dias', 'Valor_Total_Servico']
    X = df[features]
    y = df['Reclamacao_Posterior_Binario']
    X_encoded = pd.get_dummies(X, columns=['Modelo_Bomba', 'Etapas_Manutencao', 'Tecnico_Responsavel', 'Peca_Utilizada'], drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)

    scaler = StandardScaler()
    numerical_cols = ['Lead_Time_Dias', 'Valor_Total_Servico']
    X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

    dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt_model.fit(X_train, y_train)

    mlp_model = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=500, random_state=42, solver='adam')
    mlp_model.fit(X_train, y_train)

    return df, dt_model, mlp_model, X_train, X_test, y_test

# Gera os dados e modelos (apenas uma vez)
df, dt_model, mlp_model, X_train, X_test, y_test = gerar_dados_e_modelos()

# =================================================================
# 2. DEFINIÇÃO E EXIBIÇÃO DO DASHBOARD (Streamlit)
# =================================================================

st.set_page_config(layout="wide")

st.title("⚙️ Dashboard de Gestão Inteligente para Oficina Diesel")
st.markdown("---")

# --- Métricas Chave (KPIs) ---
st.header("1. Visão Geral da Gestão e Finanças")

col1, col2, col3, col4 = st.columns(4)

lead_time_medio = df['Lead_Time_Dias'].mean()
taxa_reclamacao_global = (df['Reclamacao_Posterior'] == 'Sim').sum() / len(df)
valor_medio_servico = df['Valor_Total_Servico'].mean()
servicos_em_espera = (df['Status_Final'] != 'Concluído').sum()

col1.metric(label="Tempo Médio de Reparo (TMR)", value=f"{lead_time_medio:.2f} dias", delta="Meta: 3.0 dias")
col2.metric(label="Taxa Global de Reclamação", value=f"{taxa_reclamacao_global:.2%}", delta_color="inverse", delta="Meta: 5.0%")
col3.metric(label="Valor Médio do Serviço", value=f"R$ {valor_medio_servico:,.2f}", delta="Aumento de 2% no mês")
col4.metric(label="Serviços em Processo/Espera", value=servicos_em_espera, delta_color="off", delta=f"{((df['Status_Final'] == 'Aguardando Peça').sum())} aguardando peças")

st.markdown("---")

# --- Análise de Qualidade e Justificativa ---
st.header("2. Análise de Qualidade e Redução de Reclamações")

# Gráfico 1: Desempenho dos Técnicos
df_reclamacao_tecnico = df.groupby('Tecnico_Responsavel')['Reclamacao_Posterior_Binario'].mean().reset_index()
df_reclamacao_tecnico['Reclamacao_Posterior_Binario'] = df_reclamacao_tecnico['Reclamacao_Posterior_Binario'] * 100

fig_tecnicos = px.bar(
    df_reclamacao_tecnico,
    x='Tecnico_Responsavel',
    y='Reclamacao_Posterior_Binario',
    title='Taxa Média de Reclamação por Técnico',
    labels={'Reclamacao_Posterior_Binario': 'Taxa de Reclamação (%)', 'Tecnico_Responsavel': 'Técnico'},
    color='Reclamacao_Posterior_Binario',
    color_continuous_scale=px.colors.sequential.Reds
)
st.plotly_chart(fig_tecnicos, use_container_width=True)

col_just, col_pecas = st.columns(2)

# Gráfico 2: Fatores de Reclamação (Importância da Árvore de Decisão)
feature_importances_dt = pd.Series(dt_model.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(5)
fig_importancia = px.bar(
    feature_importances_dt,
    orientation='h',
    title='Top 5 Fatores de Risco de Reclamação (Justificativa Clara)',
    labels={'value': 'Importância Relativa', 'index': 'Fator'},
)
col_just.plotly_chart(fig_importancia, use_container_width=True)

# Gráfico 3: Demanda de Peças (Organização de Estoque)
demanda_pecas = df['Peca_Utilizada'].value_counts().reset_index()
demanda_pecas.columns = ['Peca', 'Contagem']
fig_pecas = px.pie(
    demanda_pecas,
    values='Contagem',
    names='Peca',
    title='Distribuição da Demanda de Peças (Estoque)',
    hole=.3
)
col_pecas.plotly_chart(fig_pecas, use_container_width=True)

st.markdown("---")

# --- Análise Preditiva e Modernização ---
st.header("3. Previsão de Risco (Aprendizado de Máquina)")
st.caption("A Árvore de Decisão fornece regras simples (Interpretabilidade).")

col_ml, col_regras = st.columns(2)

# Exibição da Acurácia dos Modelos
acuracia_dt = accuracy_score(y_test, dt_model.predict(X_test))
acuracia_mlp = accuracy_score(y_test, mlp_model.predict(X_test))

col_ml.markdown(f"**Acurácia dos Modelos no Teste:**")
col_ml.text(f"Árvore de Decisão: {acuracia_dt:.2%} (Bom para Regras)")
col_ml.text(f"Rede Neural (MLP): {acuracia_mlp:.2%} (Bom para Previsão)")

# Exibição das Regras (Justificativa Clara)
regras_dt = export_text(dt_model, feature_names=list(X_train.columns), decimals=2, show_weights=True)
col_regras.subheader("Regras de Decisão (Árvore de Decisão)")
col_regras.code(regras_dt, language='text')

st.markdown("---")
st.markdown("Desenvolvido com Dados Sintéticos, Python e Streamlit para Aprimorar a Gestão.")


Overwriting dashboard_oficina.py


In [ ]:
from pyngrok import ngrok

# Cole seu token aqui ↓
ngrok.set_auth_token("35TBRjHm5FzAPVsnqLZ647WamAH_3pFqSJoTTe1daxtp8FBFs")
import subprocess
import time

# Porta padrão do Streamlit
PORT = 8501

# Fecha sessões antigas do ngrok
!kill $(lsof -t -i:{PORT}) 2>/dev/null

# Inicia o Streamlit em background
process = subprocess.Popen(['streamlit', 'run', 'dashboard_oficina.py', '--server.port', str(PORT), '--server.headless', 'true'])

# Aguarda o servidor iniciar
time.sleep(5)

# Cria o túnel público
public_url = ngrok.connect(PORT)
print("🔗 URL pública do dashboard:")
print(public_url)


🔗 URL pública do dashboard:
NgrokTunnel: "https://hildegarde-unexcerpted-devona.ngrok-free.dev" -> "http://localhost:8501"
